In [1]:
import pickle
import os
import json
import colorthief
from tqdm import tqdm

In [2]:
from colorthief import ColorThief

In [3]:
import webcolors

In [4]:
from scipy.spatial import KDTree

In [5]:
from webcolors import (
    CSS3_HEX_TO_NAMES,
#     css3_hex_to_names,
    hex_to_rgb,
)

In [6]:
color_compiled = {}

In [7]:
with open('css_color_to_name.p','rb') as F:
    color_to_name = pickle.load(F)

In [8]:
inverted_css3 = {}
for key,value in CSS3_HEX_TO_NAMES.items():
    inverted_css3[value] = key

In [9]:
def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index], [rgb_values[index][0],rgb_values[index][1],rgb_values[index][2]]

# Azure

In [10]:
with open('azure_detected_colors_full.p','rb') as F:
    azure_detected_colors = pickle.load(F)

In [11]:
for im_path, color_info in azure_detected_colors.items():
    im_name = im_path.split('/')[-1]
    if len(color_info) > 0:
        if color_info[0].lower() in set(color_to_name.values()):
            chosen_color = color_info[0]
        elif color_info[0] == 'Grey':
            chosen_color = 'gray'
        elif color_info[0] == 'Teal':
            chosen_color = 'green'
        else:
            print(color_info[0])
    
    if not im_name in color_compiled.keys():
        color_compiled[im_name] = {}
    
    color_compiled[im_name]['azure'] = chosen_color.lower()

# Google

In [12]:
from google.cloud import vision_v1

In [13]:
for im in tqdm(os.listdir('static/dataset/api_outputs/')):
    im_info_path = os.path.join('static/dataset/api_outputs/', im)
    with open(im_info_path,'rb') as F:
        im_info = pickle.load(F)
    best_score = 0
    for color in im_info[1]:
        if color.score > best_score:
            best_score = color.score
            best_color = color.color
    
    rgb_tuple = int(best_color.red), int(best_color.green), int(best_color.blue)
    named_color,_ = convert_rgb_to_names(rgb_tuple)
    easy_color_name = color_to_name[inverted_css3[named_color]]
    
    im_name = im.replace('p','jpg')
    
    if not im_name in color_compiled.keys():
        color_compiled[im_name] = {}
    
    color_compiled[im_name]['google'] = easy_color_name

100%|██████████| 5000/5000 [00:09<00:00, 513.91it/s]


# Histogram Method

In [14]:
with open('histogram_dominant_color.p','rb') as F:
    histogram_dominant_color = pickle.load(F)

In [15]:
for im_path, color_name in histogram_dominant_color.items():
    im_name = im_path.split('/')[-1]
    color_compiled[im_name]['histogram'] = color_name

In [16]:
# color_compiled

# Humans

In [17]:
import pandas

In [18]:
df = pandas.read_csv('../turing_test/out.csv')

In [19]:
per_image_colors = {}

In [24]:
for row in df.iterrows():
    row_datastring = row[1]['datastring']
    if isinstance(row_datastring,str):
        row_data = json.loads(row_datastring)['data']
        for i in range(3,28):
            person_data = row_data[i]['trialdata']
            image = person_data['imageID'].split('/')[-1]
            if isinstance(person_data['response4'],list):
                chosen_color = person_data['response4'][0]
            else:
                chosen_color = person_data['response4']
            color_compiled[image]['humans'] = chosen_color

False

In [47]:
color_compiled_with_humans = {}

In [48]:
for key in color_compiled.keys():
    if len(color_compiled[key].keys()) == 4:
        color_compiled_with_humans[key] = color_compiled[key]

# Write to html files

In [53]:
import pickle
from tqdm import tqdm

In [63]:
string_1 = "<center> <h4>Question: What is the dominant color in this image? <br> Speaker Response: Bello</h4> </center>"

In [64]:
agent_to_code = {}
agent_to_code['humans'] = 'a'
agent_to_code['azure'] = 'b'
agent_to_code['histogram'] = 'c'
agent_to_code['google'] = 'd'

In [65]:
os.makedirs('static/responses_color',exist_ok=True)

In [66]:
for im_name in tqdm(color_compiled_with_humans.keys()):
    for agent in color_compiled_with_humans[im_name].keys():
        agent_code = agent_to_code[agent]
        response = color_compiled_with_humans[im_name][agent]
        file_name = "static/responses_color/%s_%s_0.html"%(im_name.split('.jpg')[0],agent_code)
        string_new = string_1.replace('Bello',response.lower())
        with open(file_name,'w') as F:
            print(string_new, file = F)

100%|██████████| 873/873 [00:06<00:00, 137.68it/s]


In [67]:
all_htmls = os.listdir('static/responses_color/')

In [73]:
image_nums = [i.split('.')[0] for i in list(color_compiled_with_humans.keys())]

In [74]:
big_str = "','".join(image_nums)
print("var image_nums = ['%s']"%big_str)

var image_nums = ['1','10','100','1000','1001','1003','1004','1006','1007','1008','1010','1011','1012','1014','1015','1017','1019','102','1020','1022','1024','1025','1026','1027','1028','103','1030','1031','1032','1034','1036','1039','1040','1042','1043','1044','1045','1047','1048','1049','1051','1052','1053','1054','1055','1056','1057','1059','106','1060','1062','1063','1064','1065','1066','1067','1068','1069','107','1070','1072','1073','1074','1075','1076','1077','1078','1079','108','1080','1082','1083','1085','1087','1088','1089','109','1090','1091','1092','1093','1095','1096','1098','1100','1102','1103','1105','1108','1109','111','1111','1112','1113','1114','1115','1118','1119','112','1120','1121','1122','1124','1125','1127','1128','1129','113','1130','1131','1132','1133','1134','1135','1137','1139','114','1140','1141','1142','1145','1146','1148','1149','1150','1151','1152','1153','1154','1155','1156','1157','116','1160','1161','1162','1163','1166','1167','1168','117','1171','1172'